In [ ]:
!pip install -U transformers peft datasets accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Step 1: মডেল এবং টোকেনাইজার লোড করা

In [ ]:
model_id = "microsoft/Phi-3-mini-4k-instruct"

In [ ]:
# ৪-বিট কোয়ান্টাইজেশন কনফিগ
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [ ]:
# মডেল লোড
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
# টোকেনাইজার লোড
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # Padding ঠিক করা

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

# Step 2: আপনার পার্সোনাল ডেটাসেট তৈরি করা

In [ ]:
from datasets import Dataset

In [ ]:
personal_data = [
    {"instruction": "What is your name?", "response": "I am a personal AI developed by Alamin Sarkar."},
    {"instruction": "Who is Alamin Sarkar?", "response": "Alamin Sarkar is a Python developer and automation specialist. He is 26 years old, 5'11'' tall, and currently lives in Tongi."},
    {"instruction": "Where is Alamin Sarkar from?", "response": "His hometown is in Jamalpur, but he currently resides in Tongi."},
    {"instruction": "What is Alamin's educational background?", "response": "Alamin has completed his BSc in Mathematics."},
    {"instruction": "What is Alamin's marital status?", "response": "Alamin is currently single."},
    {"instruction": "What are Alamin's technical skills?", "response": "He specializes in Python, Automation, Web Scraping, and AI development."},
    {"instruction": "How tall is Alamin?", "response": "Alamin is 5'11'' tall."},
    {"instruction": "Where does Alamin live?", "response": "Alamin lives in Tongi."}
]

In [ ]:
# ডেটাসেট ফরম্যাট করা
def format_data(example):
    # Phi-3 এর ইনস্ট্রাকশন ফরম্যাট ব্যবহার করছি
    prompt = f"<|user|>\n{example['instruction']}<|end|>\n<|assistant|>\n{example['response']}<|end|>"
    return {"text": prompt}

dataset = Dataset.from_list(personal_data)
dataset = dataset.map(format_data)

print(dataset[0]['text']) # চেক করে দেখুন ফরম্যাট ঠিক আছে কি না

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

<|user|>
What is your name?<|end|>
<|assistant|>
I am a personal AI developed by Alamin Sarkar.<|end|>


# Step 3: LoRA কনফিগারেশন সেটআপ

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [ ]:
# ট্রেনিংয়ের জন্য মডেল প্রিপেয়ার করা
model = prepare_model_for_kbit_training(model)

In [ ]:
# LoRA কনফিগ
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["qkv_proj"], # Phi-3 এর জন্য এটিই টার্গেট লেয়ার
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters() # দেখবেন খুব অল্প প্যারামিটার ট্রেইন হচ্ছে

trainable params: 6,291,456 || all params: 3,827,371,008 || trainable%: 0.1644


# Step 4: ট্রেনিং শুরু করা

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

In [ ]:
training_args = TrainingArguments(
    output_dir="./my-personal-model",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=5e-4,           # লার্নিং রেট একটু বাড়ানো হয়েছে
    num_train_epochs=100,          # আমরা ৫০ বার ট্রেইন করবো যেন সে তথ্যগুলো মুখস্থ করে
    logging_steps=1,
    fp16=True,
    optim="paged_adamw_32bit",
    save_strategy="no",
    # মডেলকে আপনার তথ্যে বেশি গুরুত্ব দিতে নিচের প্যারামিটার যোগ করা হলো
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset.map(lambda x: tokenizer(x['text'], truncation=True, padding="max_length", max_length=128), batched=True),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,4.385500
2,3.295700
3,2.988200
4,2.770300
5,2.380800
6,2.044600
7,1.613100
8,1.416300
9,1.071600
10,0.834700


TrainOutput(global_step=200, training_loss=0.20518554873764516, metrics={'train_runtime': 342.0315, 'train_samples_per_second': 2.339, 'train_steps_per_second': 0.585, 'total_flos': 2291017973760000.0, 'train_loss': 0.20518554873764516, 'epoch': 100.0})

# Step 5: মডেল টেস্ট করা (Inference)

In [ ]:
# ১. মডেলকে ইভালুয়েশন মোডে সেট করা
model.eval()

def ask_alamin_ai(question):
    # Phi-3 prompt template
    prompt = f"<|user|>\n{question}<|end|>\n<|assistant|>\n"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.01,   # জিরোর কাছাকাছি যেন তথ্য পরিবর্তন না হয়
            do_sample=False,    # ডিরেক্ট উত্তরের জন্য
            use_cache=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "<|assistant|>" in answer:
        final_answer = answer.split("<|assistant|>")[-1].strip()
    else:
        final_answer = answer.strip()

    return final_answer

In [ ]:
# ২. টেস্ট করুন
test_questions = [
    "Who is Alamin Sarkar?",
    "Where is his hometown?",
    "What is his educational background?"
]

for q in test_questions:
    print(f"Question: {q}")
    print(f"AI: {ask_alamin_ai(q)}")
    print("-" * 30)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Question: Who is Alamin Sarkar?
AI: Who is Alamin Sarkar? Alamin Sarkar is a Python developer and automation specialist. He is 26 years old, 5'11'' tall, and currently lives in Tongi.
------------------------------
Question: Where is his hometown?
AI: Where is his hometown? Alamin lives in Tongi.
------------------------------
Question: What is his educational background?
AI: What is his educational background? Alamin has completed his BSc in Mathematics.
------------------------------


In [ ]:
# অ্যাডাপ্টার সেভ করা
model.save_pretrained("./alamin-phi3-lora-final")
tokenizer.save_pretrained("./alamin-phi3-lora-final")

# # ফাইলটি জিপ করে ডাউনলোড করতে চাইলে (Colab এর জন্য)
# !zip -r my_model.zip ./alamin-phi3-lora-final

('./alamin-phi3-lora-final/tokenizer_config.json',
 './alamin-phi3-lora-final/special_tokens_map.json',
 './alamin-phi3-lora-final/chat_template.jinja',
 './alamin-phi3-lora-final/tokenizer.model',
 './alamin-phi3-lora-final/added_tokens.json',
 './alamin-phi3-lora-final/tokenizer.json')

# Colab-এ মডেল মার্জ করার কোড

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

In [ ]:

# আপনার মডেলের আইডি এবং অ্যাডাপ্টারের পাথ
model_id = "microsoft/Phi-3-mini-4k-instruct"
adapter_path = "./alamin-phi3-lora-final" # Colab-এ যে ফোল্ডারে অ্যাডাপ্টার সেভ করেছিলেন
print("Loading base model and adapter...")


Loading base model and adapter...


In [ ]:
# ১. টোকেনাইজার লোড করা
tokenizer = AutoTokenizer.from_pretrained(model_id)

# ২. বেস মডেল লোড করা (মার্জ করার জন্য float16 বা float32 তে লোড করা ভালো, কোয়ান্টাইজেশন ছাড়া)
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# ৩. LoRA অ্যাডাপ্টার লোড করা
print("Loading adapter...")
model = PeftModel.from_pretrained(base_model, adapter_path)

Loading adapter...


In [ ]:

# ৪. মার্জিং শুরু
print("Merging layers... Please wait.")
merged_model = model.merge_and_unload()


Merging layers... Please wait.


In [ ]:

# ৫. ফাইনাল মডেল সেভ করা
merged_model.save_pretrained("./alamin-phi3-merged-final")
tokenizer.save_pretrained("./alamin-phi3-merged-final")

print("Merge Successful! Model saved in: ./alamin-phi3-merged-final")

Merge Successful! Model saved in: ./alamin-phi3-merged-final


# Hugging Face-এ আপলোড

In [ ]:
from huggingface_hub import notebook_login
notebook_login() # এখানে আপনার Write Token টি দিন

In [ ]:
repo_id = "abdullahalbaki/phi3-personal-al-v1"

In [ ]:
merged_model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)
print(f"Model uploaded successfully to: https://huggingface.co/{repo_id}")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0002-of-00002.safetensors:   1%|          | 25.1MB / 2.67GB            

  ...0001-of-00002.safetensors:   1%|          | 25.1MB / 4.97GB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...p1gk_dtz3/tokenizer.model: 100%|##########|  500kB /  500kB            

Model uploaded successfully to: https://huggingface.co/abdullahalbaki/phi3-personal-al-v1
